In [1]:
from ParameterTuning.AbstractClassSearch import DictionaryKeys as DK
from ParameterTuning.BayesianSearch import BayesianSearch
import utils 
import pandas as pd
import numpy as np
import warnings
import CollaborativeRecommenderBatch as CRB
from Base.Evaluation.Evaluator import SequentialEvaluator

In [2]:
tracks = pd.read_csv('../input/tracks.csv')
train = pd.read_csv('../input/train.csv')
#target = pd.read_csv('../input/target_playlists.csv')

In [3]:
icm_csr = utils.build_icm_csr(tracks)
urm_csr = utils.build_urm_csr(train)

C:\Users\giuse\Anaconda3\envs\tensorflow\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [4]:
train_csr, test_csr = utils.split(urm_csr, TEST_SET_HOLDOUT = 0.5)

In [5]:
indices = test_csr.getnnz(axis = 1).nonzero()[0]
test_indices = np.random.choice(indices, size = int(len(indices) / 2), replace = False)
valid_indices = np.setdiff1d(indices, test_indices, assume_unique = True)

In [6]:
evaluator_validation = SequentialEvaluator(test_csr, ignore_users = test_indices, cutoff_list = [10])
evaluator_test = SequentialEvaluator(test_csr, ignore_users = valid_indices, cutoff_list = [10])

Ignoring 9744 Users
Ignoring 9744 Users


In [7]:
dictionary = {DK.FIT_POSITIONAL_ARGS:[], 
              DK.FIT_KEYWORD_ARGS:{}, 
              DK.CONSTRUCTOR_POSITIONAL_ARGS:[], 
              DK.CONSTRUCTOR_KEYWORD_ARGS:{"URM_csr":train_csr}, 
              DK.FIT_RANGE_KEYWORD_ARGS: {'topK':[1, 10, 15, 20], 'shrink' : [0, 1, 5, 10, 15]}}

bs = BayesianSearch(CRB.CollaborativeItemBasedRecommender, 
                    evaluator_validation = evaluator_validation, 
                    evaluator_test = evaluator_test)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = bs.search(dictionary, output_root_path = "BayesianSearchOutput/", save_model = "no", n_cases = 2)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |    shrink |      topK | 
BayesianSearch: Testing config: {'topK': 1, 'shrink': 5}
Similarity column 18200 ( 88 % ), 604.06 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 605.92 column/sec, elapsed time 0.57 min
SequentialEvaluator: Processed 9744 ( 100.00% ) in 13.37 seconds. Users per second: 729
BayesianSearch: New best config found. Config: {'topK': 1, 'shrink': 5} - MAP: 0.05077273438110938



KeyboardInterrupt: 